In [40]:
import antimeridian
import geopandas as gpd
from odc.geo import XY, BoundingBox
from odc.geo.gridspec import GridSpec
from shapely.geometry import shape

In [49]:
# Create a gridspec
pacific_epsg = "EPSG:3832"
# The origin is in the projected CRS. This works for Landsat.
pacific_gridspec = GridSpec(crs=pacific_epsg, tile_shape=(3200,3200), resolution=30, origin=XY(-3000000,-4000000))

# This grid is for Sentinel-2 and has the same footprint
# pacific_gridspec_s2 = GridSpec(crs=pacific_epsg, tile_shape=(9600,9600), resolution=10, origin=XY(-3000000,-4000000))
bounds = BoundingBox(120, -30, 280, 30, crs="EPSG:4326").to_crs(pacific_epsg)

tiles = pacific_gridspec.tiles(bounds)

geometry, index = zip(*[(a_tile[1].boundingbox.polygon.geom, a_tile[0]) for a_tile in tiles])

# GeoSeries in native projection, should not have rounding issues like the geojson version
# Also sets series index by tile row, column so will work with existing code elsewhere
gs = gpd.GeoSeries(geometry, index, crs=pacific_epsg)


In [42]:
pacific_admin = gpd.read_file("data/pacific_admin.gpkg")
admin_projected = pacific_admin.to_crs(pacific_epsg)
intersected = gpd.sjoin(gpd.GeoDataFrame(geometry=gs), admin_projected)

In [47]:
grid = intersected.rename(columns={"ISO2":"country_code"}).reset_index(names=["row", "column"])

grid["tile_id"] = grid.apply(lambda x: f"{x.row},{x.column}", axis=1)
grid["bbox"] = grid["geometry"].apply(lambda x: ",".join([f"{b:6.0f}" for b in x.bounds]))
grid_3832 = grid.drop(columns=["row", "column", "index_right", "NAME", "osm_id", "layer"])

grid_4326 = grid
grid_4326.geometry = grid.to_crs(4326).geometry.apply(lambda geom: shape(antimeridian.fix_shape(geom)))
grid_4326.explore()


/home/jesse/Projects/D4D/dep-grid/venv/lib/python3.10/site-packages/antimeridian/_implementation.py:391: FixWindingWarning: The exterior ring of this shape is wound clockwise. Since this is a common error in real-world geometries, this package is reversing the exterior coordinates of the input shape before running its algorithm. If you know that your input shape is correct (i.e. if your data encompasses both poles), pass `fix_winding=False`.
  FixWindingWarning.warn()
/home/jesse/Projects/D4D/dep-grid/venv/lib/python3.10/site-packages/antimeridian/_implementation.py:534: FixWindingWarning: The exterior ring of this shape is wound clockwise. Since this is a common error in real-world geometries, this package is reversing the exterior coordinates of the input shape before running its algorithm. If you know that your input shape is correct (i.e. if your data encompasses both poles), pass `fix_winding=False`.
  FixWindingWarning.warn()


In [48]:
grid_4326.to_file("grid_pacific.geojson")
grid_3832.to_file("grid.gpkg")